In [1]:
import sqlite3
import pandas as pd

In [2]:
# Connect to the SQLite database
conn = sqlite3.connect('social_network_anonymized.db')

cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
table_name = cursor.fetchall()

print (table_name)

# Read the entire database into a pandas DataFrame
media = pd.read_sql_query(f"SELECT * FROM Media", conn)
profile_activity = pd.read_sql_query(f"SELECT * FROM ProfileActivity", conn)
activity_media = pd.read_sql_query(f"SELECT * FROM ActivityMedia", conn)
profile_connection = pd.read_sql_query(f"SELECT * FROM ProfileConnection", conn)
profiles = pd.read_sql_query(f"SELECT * FROM Profiles", conn)
activity = pd.read_sql_query(f"SELECT * FROM Activity", conn)

# Close the connection
conn.close()

[('Media',), ('ProfileActivity',), ('ActivityMedia',), ('ProfileConnection',), ('Profiles',), ('Activity',)]


In [29]:
def pair_profile_activities(profile_activity_df):
    sources = profile_activity_df[profile_activity_df.relationship_type == "source"]
    targets = profile_activity_df[profile_activity_df.relationship_type == "target"]
    
    merged = pd.merge(
        sources,
        targets,
        on="activity_id",
        suffixes=("_source", "_target")
    )

    return merged[[
        "profile_id_source",
        "profile_id_target",
        "activity_id"
    ]]

def merge_activities(df, activity_df, left_on):
    return pd.merge(
        df,
        activity_df,
        left_on=left_on,
        right_on="id"
    )

In [35]:
profile_pairs = pair_profile_activities(profile_activity)
profile_pairs_activity = merge_activities(profile_pairs, activity, left_on="activity_id")

In [36]:
profile_pairs_activity

,profile_id_source,profile_id_target,activity_id,id,type,timestamp,content,description,platform
0,27059,24703,1,1,updated-friends-list-on-facebook,1734533582000,,Michael Taylor 53 connected with John Rodrigue...,facebook
1,27059,36779,2,2,updated-friends-list-on-facebook,1734533582000,,Michael Taylor 53 connected with Robert Martin...,facebook
2,27059,8299,3,3,updated-friends-list-on-facebook,1734533582000,,Michael Taylor 53 connected with Jessica Taylor 8,facebook
3,27059,18673,4,4,updated-friends-list-on-facebook,1734533582000,,Michael Taylor 53 connected with Sakei Exopets,facebook
4,27059,14731,5,5,updated-friends-list-on-facebook,1734533582000,,Michael Taylor 53 connected with William Marti...,facebook
...,...,...,...,...,...,...,...,...,...
23304,27059,16719,124975,124975,updated-friends-list-on-facebook,1735743735000,,Michael Taylor 53 connected with Thomas Willia...,facebook
23305,10786,6144,125037,125037,updated-friends-list-on-facebook,1735744281000,,Barbara Jones 5 connected with John Smith 10,facebook
23306,10786,6144,125038,125038,updated-friends-list-on-facebook,1734534129000,,Barbara Jones 5 connected with John Smith 10,facebook
23307,10786,815,125070,125070,updated-friends-list-on-facebook,1735744281000,,Barbara Jones 5 connected with Michael Garcia 2,facebook
